# Data Management básico con pandas

Sirve para tener en un vistazo las tareas más habituales que realizamos en el día a día con Pandas. Para aquel que se esté introduciendo al uso de Python puede ser de utilidad tener todo junto y más claro, a mi personalmente me sirve para no olvidar cosas que ya no uso. En una sola entrada recogemos las dudas más básicas cuando nos estamos iniciando con Python. Las tareas más comunes son:

* Seleccionar columnas
* Eliminar columnas
* Seleccionar registros
* Crear nuevas variables
* Sumarizar datos
* Ordenar datos

Se va a emplear el conjunto de datos iris y que nos descargamos directamente de una url para ello las primeras sentencias que hemos de ejecutar son las siguientes:

In [2]:
import pandas as pd
import io
import requests
url='https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

Se pretende en pocas líneas y en un sólo notebook recoger las tareas más habituales en el manejo de datos. 

## Seleccionar columnas

Directamente

In [4]:
#Directamente
df2 = df[['sepal_length','sepal_width']]
df2.head()

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


Mediante una lista. Método recomendado.

In [5]:
seleccionadas = ['sepal_length','sepal_width']
df2 = df[seleccionadas]
df2.head()

,sepal_length,sepal_width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


## Eliminar columnas

In [6]:
df3 = df.drop(columns=['sepal_length','sepal_width'])
df3.head()

,petal_length,petal_width,species
0,1.4,0.2,setosa
1,1.4,0.2,setosa
2,1.3,0.2,setosa
3,1.5,0.2,setosa
4,1.4,0.2,setosa


## Seleccionar registros

In [9]:
#value_counts() nos permite hacer de un modo rápido una tabla de frecuencias
df['species'].value_counts()

versicolor    50
virginica     50
setosa        50
Name: species, dtype: int64

Registros que cumplan una condición.

In [11]:
df4 = df[df['species']=="setosa"]
df4['species'].value_counts()

setosa    50
Name: species, dtype: int64

In [12]:
df4 = df[df['species'] != "setosa"]
df4['species'].value_counts()

versicolor    50
virginica     50
Name: species, dtype: int64

Hay que tener en cuenta el modo en el que pandas realiza las condiciones más complejas. 

In [15]:
df5 = df.loc[(df.sepal_length<5) & (df.species=="setosa")]
df5.species.value_counts()

setosa    20
Name: species, dtype: int64

In [19]:
df6 = df[((df['sepal_length']>5) | (df['sepal_length']<2)) & (df['species'] != "setosa")]
df6.species.value_counts()

virginica     49
versicolor    47
Name: species, dtype: int64

Emplearemos <code>isin</code> para hacer selecciones de lista, del mismo modo que hacemos in ()

In [20]:
lista = ['setosa', 'virginica']
df7 = df[df['species'].isin(lista)]
df7['species'].value_counts()

virginica    50
setosa       50
Name: species, dtype: int64

## Creación de nuevas variables con Python

In [21]:
# Corregimos una variable por su media
df['sepal_length_tipi'] = df['sepal_length']/df['sepal_length'].mean()
df['sepal_length_tipi'].describe()

count    150.000000
mean       1.000000
std        0.141711
min        0.735881
25%        0.872790
50%        0.992584
75%        1.095265
max        1.351968
Name: sepal_length_tipi, dtype: float64

Para crear nuevas variables hay una función muy común que es <code>where</code> de numpy que nos permite evaluar condiciones:

In [22]:
import numpy as np

df['sepal_length_altas'] = np.where(df['sepal_length'] > np.mean(df['sepal_length']),
                                    "Por encima de la media", "Por debajo de la media")
df['sepal_length_altas'].value_counts()

Por debajo de la media    80
Por encima de la media    70
Name: sepal_length_altas, dtype: int64

## Sumarizar datos

La forma más rápida y directa

In [25]:
df[['sepal_length','species']].groupby('species').mean()

,sepal_length
species,
setosa,5.006
versicolor,5.936
virginica,6.588


In [26]:
df[['sepal_length','species']].groupby('species').count()

,sepal_length
species,
setosa,50
versicolor,50
virginica,50


Múltiples columnas.

In [33]:
df.groupby(['species','sepal_length_altas']).min()

sepal_length  sepal_width  petal_length  \
species    sepal_length_altas                                                
setosa     Por debajo de la media           4.3          2.3           1.0   
versicolor Por debajo de la media           4.9          2.0           3.0   
           Por encima de la media           5.9          2.2           4.0   
virginica  Por debajo de la media           4.9          2.5           4.5   
           Por encima de la media           5.9          2.2           4.8   

                                   petal_width  sepal_length_tipi  
species    sepal_length_altas                                      
setosa     Por debajo de la media          0.1           0.735881  
versicolor Por debajo de la media          1.0           0.838562  
           Por encima de la media          1.0           1.009698  
virginica  Por debajo de la media          1.7           0.838562  
           Por encima de la media          1.4           1.009698

Para estas sumarizaciones es muy común usar la función <code>crosstab</code>, veamos algunos ejemplos de uso.

In [39]:
pd.crosstab(df['species'],df['sepal_length_altas'])

sepal_length_altas,Por debajo de la media,Por encima de la media
species,,
setosa,50,0
versicolor,24,26
virginica,6,44


In [40]:
pd.crosstab(df['species'],df['sepal_length_altas'], margins=True, margins_name="Total")

sepal_length_altas,Por debajo de la media,Por encima de la media,Total
species,,,
setosa,50,0,50
versicolor,24,26,50
virginica,6,44,50
Total,80,70,150


In [42]:
pd.crosstab(df['species'],df['sepal_length_altas'], values=df['sepal_length'], aggfunc='mean')

sepal_length_altas,Por debajo de la media,Por encima de la media
species,,
setosa,5.006,NaN
versicolor,5.500,6.338462
virginica,5.600,6.722727


In [31]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length_tipi,sepal_length_altas
0,5.1,3.5,1.4,0.2,setosa,0.872790,Por debajo de la media
1,4.9,3.0,1.4,0.2,setosa,0.838562,Por debajo de la media
2,4.7,3.2,1.3,0.2,setosa,0.804335,Por debajo de la media
3,4.6,3.1,1.5,0.2,setosa,0.787222,Por debajo de la media
4,5.0,3.6,1.4,0.2,setosa,0.855676,Por debajo de la media
